In [49]:
# Cell 1: Install Dependencies
!pip install scikit-learn torch pandas numpy

# Cell 2: Import Libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler

# Cell 3: Load Data (Replace with actual dataset path)
data = pd.read_csv('laptops.csv', encoding='ISO-8859-1')
data = data[['name', 'price(in Rs.)', 'ram', 'storage', 'display(in inch)', 'rating']]
data.dropna(inplace=True)

In [51]:
# Cell 4: Generate Synthetic User IDs
data['user_id'] = range(len(data))
data['rating'] = np.random.randint(1, 6, data.shape[0])  # Simulated ratings from 1-5




In [53]:
# Function to extract numeric values from RAM and Storage
def extract_numeric(value):
    numbers = re.findall(r'\d+', str(value))  # Extract digits
    return int(numbers[0]) if numbers else 0  # Convert to int

# Apply to relevant columns
data['ram'] = data['ram'].apply(extract_numeric)
data['storage'] = data['storage'].apply(extract_numeric)
data['display(in inch)'] = pd.to_numeric(data['display(in inch)'], errors='coerce')  # Convert safely

# Normalize numerical features
scaler = MinMaxScaler()
data[['price(in Rs.)', 'ram', 'storage', 'display(in inch)']] = scaler.fit_transform(
    data[['price(in Rs.)', 'ram', 'storage', 'display(in inch)']]
)

# Check the processed data
print(data.head())

                                       name  price(in Rs.)       ram  \
0             Lenovo Intel Core i5 11th Gen       0.142684  0.428571   
1            Lenovo V15 G2 Core i3 11th Gen       0.065301  0.142857   
2      ASUS TUF Gaming F15 Core i5 10th Gen       0.103218  0.142857   
3  ASUS VivoBook 15 (2022) Core i3 10th Gen       0.054645  0.142857   
4                   Lenovo Athlon Dual Core       0.009107  0.000000   

    storage  display(in inch)  rating  user_id  
0  1.000000          0.170940       2        0  
1  0.000000          0.170940       4        1  
2  1.000000          0.170940       5        2  
3  1.000000          0.170940       3        3  
4  0.499022          0.102564       5        4  


In [54]:
class MatrixFactorization(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=10):
        super().__init__()
        self.user_emb = nn.Embedding(num_users, embedding_dim)
        self.item_emb = nn.Embedding(num_items, embedding_dim)
        self.fc = nn.Linear(embedding_dim, 1)

    def forward(self, user, item):
        user_embedded = self.user_emb(user)
        item_embedded = self.item_emb(item)
        interaction = user_embedded * item_embedded
        return self.fc(interaction).squeeze()


In [64]:
import torch
import torch.nn as nn

class LaptopRecommendationModel(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim=10):
        super(LaptopRecommendationModel, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        self.fc = nn.Linear(embedding_dim, 1)  # Output single score

    def forward(self, user_ids, item_ids):
        user_embeds = self.user_embedding(user_ids)
        item_embeds = self.item_embedding(item_ids)
        interaction = user_embeds * item_embeds  # Element-wise multiplication
        score = self.fc(interaction).squeeze()
        return score


In [65]:
data['laptop_id'] = range(len(data))  # Assigns a unique ID to each laptop
num_items = data['laptop_id'].max() + 1  # Ensure it matches the max index
model = LaptopRecommendationModel(num_users, num_items, embedding_dim=10)



In [71]:
def recommend_laptops(laptop_name, top_n=5):
    try:
        item_index = data[data['name'] == laptop_name].index[0]
        item_tensor = torch.tensor([item_index], dtype=torch.long)
        similarities = model.item_emb.weight @ model.item_emb(item_tensor).T
        top_indices = similarities.squeeze().argsort(descending=True)[:top_n]
        return data.iloc[top_indices][['name', 'price(in Rs.)', 'ram', 'storage', 'display(in inch)', 'rating']]
    except IndexError:
        return 'Laptop not found in dataset.'

# Example Usage
recommend_laptops("Lenovo Intel Core i5 11th Gen", top_n=5)

,name,price(in Rs.),ram,storage,display(in inch),rating
0,Lenovo Intel Core i5 11th Gen,0.142684,0.428571,1.000000,0.170940,2
422,HP Victus Ryzen 5 Hexa Core 5600H,0.163934,0.142857,1.000000,0.170940,2
347,Lenovo IdeaPad 3 Core i3 11th Gen,0.068940,0.142857,1.000000,0.170940,4
44,ASUS VivoBook 14 Pentium Silver,0.030358,0.000000,0.499022,0.102564,5
973,MSI Core i5 12th Gen,0.174560,0.142857,1.000000,0.170940,1
